In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as BS
import matplotlib.pyplot as plt
import numpy as np
import re

In [ ]:
rawplant = pd.read_csv('../slcapstone/data/plantInfo-raw .csv')

In [ ]:
rawplant['Name'] = rawplant['Name'].str.split(',', expand = True)[0]
rawplant['Name'] = rawplant['Name'].str.split('-', expand = True)[0]

In [ ]:
rawplant = rawplant.drop(columns =['culinaryHints','culinaryPreservation', 'url'])

In [ ]:
rawplant['growthType'] = rawplant['sowInstructions'].apply(lambda x: 'Easy to grow' if 'Easy to grow' in x else 'Normal/Hard')

In [ ]:
import re


def extract_format(text):
    match = re.search(r'(\d+\s*(?:-\s*\d+)?\s*(weeks|months|years)?)', text)
    if match:
        return match.group(0)
    else:
        return None


rawplant['harvestInstructions'] = rawplant['harvestInstructions'].apply(extract_format)

In [ ]:
compatible_plants_split = rawplant['compatiblePlants'].str.split(': ', expand=True)
compatcolumn = len(compatible_plants_split.columns)


compatible_plants_split.columns = [f'compatiblePlant_{i}' for i in range(compatcolumn)]
rawplant = rawplant.drop(columns=['compatiblePlants'])

rawplant = pd.concat([rawplant, compatible_plants_split], axis=1)
rawplant

In [ ]:
rawplant.columns

In [ ]:
rawplant.drop(columns = ['compatiblePlant_0'])

In [ ]:
rawplant = rawplant.rename(columns = {'Name':'plants'})
rawplant

In [ ]:
rawplant['spaceInstructions'] = rawplant['spaceInstructions'].str.split(':').str[-1].str.strip()
rawplant

In [ ]:
rawplant[~rawplant['spaceInstructions'].str.contains('inches', na=False)]

In [ ]:
rawplant['spaceInstructions'] = rawplant['spaceInstructions'].replace({
    'Rows 1 Metre apart': '39',
    '5 - 8 cm apart': '3',
    '20 - 30 cm apart': '11',
    '15 cm apart': '6',
    '12 - 20 cm apart': '8',
    'close together': '2',
    '140 cm apart': '55',
    'Thin to 18 inches': '18',
    'Thin to 12 inches': '12'
})

In [ ]:
rawplant['spaceInstructions'] = rawplant['spaceInstructions'].str.replace(' inches apart', '')
rawplant

In [ ]:
def extract_last_number(instruction):
    match = re.search(r'(\d+)\s*-\s*(\d+)$', instruction)
    if match:
        return match.group(2) 
    else:
        return instruction

rawplant['spaceInstructions'] = rawplant['spaceInstructions'].apply(extract_last_number)
rawplant

In [ ]:
rawplant[~rawplant['harvestInstructions'].str.contains('weeks', na=False)]

In [ ]:
rawplant['harvestInstructions'] = rawplant['harvestInstructions'].replace({
    '30-45': '45',
    '18 months': '78',
    '2-3 years': '104',
    '35-50': '50',
    '30-40': '40',
    '1 year': '52',
    '1 year': '52',
    '21-35': '35',
    '1 year': '52',
    '18 months': '78',
    '1 years': '52'
})

In [ ]:
rawplant['harvestInstructions'] = rawplant['harvestInstructions'].str.replace('weeks', '')

In [ ]:
def extract_last_number(instruction):
    match = re.search(r'(\d+)\s*-\s*(\d+)$', instruction)
    if match:
        return match.group(2) 
    else:
        return instruction

rawplant['harvestInstructions'] = rawplant['harvestInstructions'].apply(extract_last_number)
rawplant

In [ ]:
rawplantbetter = rawplant
rawplantbetter

In [ ]:
rawplantbetter.to_csv('rawplantbetter.csv', index = False)

In [ ]:
rawplant['harvest_time'] = rawplant['harvestInstructions'].str.extract(r'(\d+)').astype(float)

In [ ]:
rawplant.to_csv('rawplant.csv', index = False)

In [ ]:
rawplant.to_csv('rawimage.csv', index = False)

In [ ]:
rawplant.to_csv('rawimage2.csv', index = False)

In [ ]:
pests = pd.read_csv('../slcapstone/data/pests.csv')
pests.to_csv('pests.csv', index = False)

In [ ]:
rawpest = pd.merge(rawplant, pests, on='plants')
rawpest

In [ ]:
rawpest.to_csv('rawpest.csv', index = False)